<a href="https://colab.research.google.com/github/ShashankBagda/Project--AI-Handwriting_to_CNC/blob/main/AI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Handwriting to CNC***

# ***Artificial Intelligence Project***

> Shashank Bagda

> 92100133020

> 6 TK1 A

Downloading Dependencies

In [1]:
!pip install opencv-python
!pip install numpy
!pip install Pillow
!pip install python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.5 MB/s eta 0:00:00


Importing all the Libraries

In [2]:
import cv2
import os
import numpy as np
import subprocess
import shutil
import time

from PIL import Image

from docx import Document
from docx.shared import Inches

Delete the Existing Images

In [3]:
def delete_files_in_folder(folder_path):
    try:
        # List all files in the specified folder
        files = os.listdir(folder_path)

        # Iterate over each file and delete it
        for file in files:
            file_path = os.path.join(folder_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Deleted file: {file_path}")

        print("All files in the folder have been deleted.")
    except Exception as e:
        print(f"An error occurred: {e}")

Find the Squares from the Input Image

In [4]:
def find_squares(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 50, 150)

    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    squares = []
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        if len(approx) == 4:
            squares.append(approx)
    return squares

Save the Extracted Images

In [5]:
def extract_and_save_squares(image_path, border_margin=5):
    image = cv2.imread(image_path)
    squares = find_squares(image)
    for i, square in enumerate(squares):
        x, y, w, h = cv2.boundingRect(square)
        # Add margin to remove the border
        x += border_margin
        y += border_margin
        w -= 2 * border_margin
        h -= 2 * border_margin
        square_img = image[y:y+h, x:x+w]
        cv2.imwrite(f'Output/box_{i}.png', square_img)

Remove the Background from Images

In [6]:
def remove_white_background(image_path, output_path):
    image = Image.open(image_path)
    image = image.convert("RGBA")

    data = image.getdata()

    new_data = []
    for item in data:
        if item[0] > 200 and item[1] > 200 and item[2] > 200:
            new_data.append((255, 255, 255, 0))  # Set the pixel to transparent
        else:
            new_data.append(item)

    # Update the image with the new data
    image.putdata(new_data)

    # Save the image with a transparent background
    image.save(output_path, "PNG")

Process Images in Folder for Removing the Background

In [7]:
def process_images_in_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            remove_white_background(input_path, output_path)


Function to read text from a Docx file

In [8]:
def read_text_from_docx(file_path):
    doc = Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

Mapping special characters and alphabets

In [9]:
special_chars = {
    '_semicolon': ';',
    '_full_stop': '.',
    '_double_InComma': '"',
    '_underscore': '_',
    '_equals': '=',
    '_addition': '+',
    '_substraction': '-',
    '_brac_curl_end': '}',
    '_brac_curl_start': '{',
    '_brac_square_end': ']',
    '_brac_square_start': '[',
    '_multiplication': '*',
    '_hashtag': '#',
    '_and': '&',
    '_percentage': '%',
    '_dollar': '$',
    '_tilde': '~',
    '_brac_round_end': ')',
    '_brac_round_start': '(',
    '_greaterthan': '>',
    '_lessthan': '<',
    '_slash': '/',
    '_toPower': '^',
    '_atharet': '@',
    '_question': '?',
    '_exclamation': '!',
    '_colon': ',',
    '_comma': ',',
    '_apostrophe': "'",
    '_backquote': '`',
    '_backSlash': '\\'
}

capital_char = {
    '_A': 'A',
    '_B': 'B',
    '_C': 'C',
    '_D': 'D',
    '_E': 'E',
    '_F': 'F',
    '_G': 'G',
    '_H': 'H',
    '_I': 'I',
    '_J': 'J',
    '_K': 'K',
    '_L': 'L',
    '_M': 'M',
    '_N': 'N',
    '_O': 'O',
    '_P': 'P',
    '_Q': 'Q',
    '_R': 'R',
    '_S': 'S',
    '_T': 'T',
    '_U': 'U',
    '_V': 'V',
    '_W': 'W',
    '_X': 'X',
    '_Y': 'Y',
    '_Z': 'Z',
}

# ***Driver Code***

Main.py

In [11]:
if __name__ == "__main__":

    folder_path = "/content/Output"
    delete_files_in_folder(folder_path)

    image_path = '/content/Untitled Folder/digital.png'
    extract_and_save_squares(image_path)

    print("Images Extracted Successfully...")
    # time.sleep(3)

    # subprocess.run(['python3', 'removebg.py'])
    # time.sleep(3)

    # subprocess.run(['python3', 'rename.py'])
    # time.sleep(3)

    # subprocess.run(['python3', 'embedder.py'])


All files in the folder have been deleted.
Images Extracted Successfully...


removebg.py

In [12]:
input_folder = "/content/Output"
output_folder = input_folder

process_images_in_folder(output_folder, output_folder)

print("Background Removed Successfully...")
# shutil.rmtree(input_folder)

Background Removed Successfully...


rename.py

In [13]:
default = 'box_'
folder_path = '/content/Output'

names = ['_full_stop','_double_InComma','_underscore','_equals','_addition','_substraction','_brac_curl_end','_brac_curl_start','_brac_square_end','_brac_square_start','_multiplication','_hashtag','_and','_percentage','_dollar','_tilde','_brac_round_end','_brac_round_start','_greaterthan','_lessthan','_slash','_toPower','_atharet','_question','_exclamation','_colon','_comma','_apostrophe','_backquote','_semicolon','9', '8', '7', '6', '5', '4', '3', '2', '1', '0','z','y', 'x', 'w', 'v', 'u', 't', 's', 'r', 'q', 'p', 'o', 'n', 'm', 'l', 'k', 'j', 'i', 'h', 'g', 'f', 'e', 'd', 'c', 'b', 'a', '_Z', '_Y', '_X', '_W', '_V', '_U', '_T', '_S', '_R', '_Q', '_P', '_O', '_N', '_M', '_L', '_K', '_J', '_I', '_H', '_G', '_F', '_E', '_D', '_C', '_B', '_A']
# print(len(names))

file_list = os.listdir(folder_path)
for i, filename in enumerate(file_list):

    old = '/content/Output/' + default + str(i) + '.png'
    new = '/content/Output/' + names[i] + '.png'
    os.rename(old, new)

print("Renamed Successfully...")

Renamed Successfully...


embedder.py

In [15]:
# input_docx = "Input.docx"
# text = read_text_from_docx(input_docx)
# output_doc = Document()

# for char in text:
#     if char == '\n':
#         #file_name = "Output/Enter.png"
#         continue
#     elif char == ' ':
#         file_name = "space.png"
#     elif char in special_chars.values():
#         for special_char, filename in special_chars.items():
#             if char == filename:
#                 file_name = f"/content/Output/{special_char}.png"
#                 break
#     for capital_key, capital_value in capital_char.items():
#         if char == capital_value:
#             file_name = f"/content/Output/{capital_key}.png"
#             break

#     else:
#         file_name = f"/content/Output/{char}.png"

#     output_doc.add_picture(file_name, width=Inches(1))

# output_doc.save("Output.docx")

# print("Words Embedded Successfully...")